In [2]:
from deepmodel_util import *
from deepmodel_definition import *
import deepmodel_util
import deepmodel_definition

deepmodel_util.num_labels = 19#
deepmodel_definition.num_labels = 19#
deepmodel_definition.emb_dim    = 300
deepmodel_definition.hidden_dim = 200

deepmodel_util.max_length_title = 30
deepmodel_util.max_length_body  = 100

deepmodel_definition.max_length_title = 30
deepmodel_definition.max_length_body  = 100

deepmodel_definition.filter_num = 200
deepmodel_definition.dropout_rate=0.5
process_setting(low =False,old = True,stop = False)

suffix = "180days_all_shuffled"
prefix = "/home/t-haohu/IndustryClassifier/Data/"


data_train = "{}/middle/train_{}.txt".format(prefix,suffix)
data_test = "{}/middle/test_{}.txt".format(prefix,suffix)


data_title_vocab    = "{}/ready/title_{}.wl".format(prefix,suffix)
data_industry_vocab = "{}/ready/industry_{}.wl".format(prefix,suffix)


title_dict =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_title_vocab).readlines()])}
industry_dict =  { x:i for i,x in enumerate([x.strip("\n") for x in open(data_industry_vocab).readlines()])}




deepmodel_util.input_xt = C.input_variable(shape=(deepmodel_util.max_length_title))
#deepmodel_util.input_xt = C.input_variable(**Sequence[Tensor[1]])
deepmodel_util.input_y  = C.input_variable(shape=(1))

deepmodel_definition.input_xt_one_hot = C.one_hot(deepmodel_util.input_xt, num_classes=len(title_dict)   ,  sparse_output=True)
deepmodel_definition.input_y_one_hot = C.one_hot(deepmodel_util.input_y  , num_classes=len(industry_dict) ,  sparse_output=True)
deepmodel_util.input_xt_one_hot = deepmodel_definition.input_xt_one_hot
deepmodel_util.input_y_one_hot = deepmodel_definition.input_y_one_hot

deepmodel_util.test_data  = load_data_static(data_test,title_dict,industry_dict)
deepmodel_util.train_data = load_data_static(data_train,title_dict,industry_dict)

#embedding = load_embedding(data_title_sample,"word2vec.model")


In [ ]:
train(create_model_rcnn_normal(),deepmodel_util.train_data,num_epochs=20,learning_rate=[5e-4*150]*2+[1e-4*150],batch_size = 150,tag = "cnn")


In [ ]:
train(create_model_cnn(),deepmodel_util.train_data,num_epochs=20,learning_rate=[5e-4*150]*2+[1e-4*150],batch_size = 150,tag = "cnn")


In [4]:
from cntk import load_model
from data_processor import *
process_setting(low=False,old = True,stop = False)
deepmodel_util.batch_size = 20
deepmodel_util.input_xt = C.input_variable(**Sequence[Tensor[1]])
    
model_file ="model/180days_all_shuffled/cnn_acc0.854.dnn"
suffix_file = "180days_all_shuffled"
industry_file = "180days_all_shuffled"

model = load_model(model_file)

data_industry_vocab = "{}/ready/industry_{}.wl".format(prefix,industry_file)
data_title_vocab    = "{}/ready/title_{}.wl".format(prefix,suffix_file)
data_body_vocab     = "{}/ready/body_{}.wl".format(prefix,suffix_file)

title_dict =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_title_vocab).readlines()])}

inference(model,"Data/middle/1day_measure_sample_valid.txt","val/dyn_cnn_1day_measure_{}.txt".format(suffix),title_dict,data_industry_vocab)



data_size:  361 batch_size:  20 num_batches_per_epoch:  19


In [7]:
#test
from cntk import load_model

#print(num_labels)
process_setting(low =False,old = True,stop = False)
deepmodel_util.num_labels=deepmodel_util.num_labels+1
model = load_model("model/180days_all_shuffled/lstm_acc0.858.dnn")
data_test_sample_body = "1day_sample_gt.txt"
test_data  = load_data_dynamic(data_test_sample_body,title_dict,industry_dict,raw=True)
test(200,model,test_data)
print("OK")

data_size:  361 batch_size:  200 num_batches_per_epoch:  2


ValueError: dtype object is not supported. If this is a batch of sequences, you need to pass them as a pure-Python list of NumPy arrays